In [1]:
%pip install --no-cache-dir faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 147.1 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install --no-cache-dir rank-bm25 snowballstemmer faiss-cpu > /dev/null
%pip install --no-cache-dir sentence-transformers > /dev/null

Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
libcugraph-cu12 25.6.0 requires libraft-cu12==25.6.*, but you have libraft-cu12 25.2.0 which is incompatible.
pylibcugraph-cu12 25.6.0 requires pylibraft-cu12==25.6.*, but you have pylibraft-cu12 25.2.0 which is incompatible.
pylibcugraph-cu12 25.6.0 requires rmm-cu12==25.6.*, but you have rmm-cu12 25.2.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install -U FlagEmbedding

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.9/163.9 kB 4.1 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 866.1/866.1 kB 19.7 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.4/134.4 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 37.6 MB/s eta 0:00:00:00:0100:01
  Created wheel for FlagEmbedding: filename=FlagEmbedding-1.3.5-py3-none-any.whl size=233746 sha256=c732fca82ae75ba6308a8e9d7916036a6187e5063c15f1d4a81edd0489f41d8f
  Stored in directory: /root/.cache/pip/wheels/fc/1c/66/c9c846a8f8cbd9574db8d76b0a61410a087bc07d53682a54f4
  Created wheel for warc3-wet-clueweb09: filename=warc3_wet_clueweb09-0.2.5-py3-none-any.whl size=18919 sha2

In [4]:
# alfabank_kaggle.py
# One-file, GPU-optimized retrieval+rerank pipeline for Kaggle
# Produces submit.csv with columns: q_id, web_list (JSON array of doc ids)

import os, sys, re, math, json, time, csv, logging, pickle, gc
from dataclasses import dataclass
from io import StringIO
from typing import List, Dict, Tuple, Optional, Iterable

import numpy as np
import pandas as pd

# ----------------------------
# Config (edit here if needed)
# ----------------------------

PROJECT_ROOT = os.getcwd()

import kagglehub
from pathlib import Path

kaggle_path = kagglehub.dataset_download("denismuradyan/alfabank-hack")
kaggle_path = Path(kaggle_path)

WEBSITES_CSV  = str(kaggle_path / "websites_updated.csv")
QUESTIONS_CSV = str(kaggle_path / "questions_clean.csv")

ARTIFACTS_DIR = os.path.join(PROJECT_ROOT, "artifacts")
os.makedirs(ARTIFACTS_DIR, exist_ok=True)
BM25_INDEX_PATH = os.path.join(ARTIFACTS_DIR, "bm25_index.pkl")

# Embeddings model (multi-lingual, strong for RU)
EMBEDDING_MODEL = "intfloat/multilingual-e5-large"
TRUST_REMOTE_CODE = True
EMBEDDING_NORMALIZE = True

# Token-based chunking
CHUNK_TOKENS   = 256
OVERLAP_TOKENS = 96

# Dense index/search
USE_FAISS = True  # FAISS cosine (via inner product on L2-normalized vecs)
FAISS_CPU = False  # force CPU for FAISS even if GPU available (debug)
BATCH_ADD_SIZE = 256  # how many chunks to embed+add per step

# BM25
USE_BM25 = True
BM25_STEMMING = True
BM25_MIN_TOKEN_LEN = 2
BM25_NORMALIZE_E = True
BM25_SYNONYMS: Dict[str, List[str]] = {
    # canonical : variants (stem will apply)
    "кредит": ["кредиты", "кредитка", "карткредит"],
    "премиум": ["a-club", "аклуб", "a club", "премиальный"],
}

# Fusion & heuristics
WEIGHT_DENSE = 0.5
WEIGHT_BM25  = 0.5
TITLE_BOOST  = 0.10  # *1.10 if query tokens ∩ title tokens ≠ ∅

URL_POSITIVE_PATTERNS = [
    r"/a-?club", r"\bpremium\b", r"wealth", r"privilege", r"/investment",
]
URL_NEGATIVE_PATTERNS = [
    r"/vacanc", r"/news", r"/press", r"/cookies", r"/privacy",
]
URL_POSITIVE_BOOST   = 0.07
URL_NEGATIVE_PENALTY = 0.10

# Dynamic weight policy
USE_DYNAMIC_WEIGHTS = True
SHORT_QUERY_TOKENS  = 5
BM25_BONUS_SHORT       = 0.25
BM25_BONUS_HAS_DIGITS  = 0.15
BM25_BONUS_KEYWORDS    = ["счет", "карта", "перевод", "платеж", "ипотека", "кредит", "валюта", "комисси"]

# Search fanouts / pooling
SEARCH_TOP_K_CHUNKS_DENSE = 200
SEARCH_TOP_K_CHUNKS_BM25  = 320
AGG_TOP_K_PER_DOC         = 6
FINAL_TOP_K_DOCS          = 5

# Query batch sizes
QUERY_EMB_BATCH_SIZE = 256
BATCH_ANSWER_CHUNK   = 128   # how many queries per batch in batch_answer()

# Cross-encoder reranker
USE_RERANKER = True
RERANKER_MODEL = "BAAI/bge-reranker-v2-gemma"
RERANK_CANDIDATE_DOCS = 150
RERANKER_BATCH_SIZE   = 32
RERANKER_MAX_LENGTH   = 512
RERANKER_ALPHA        = 0.7  # blend weight with hybrid score
RERANKER_MAX_CHARS_PER_DOC = 3000
RERANKER_CHUNKS_PER_DOC    = 4

# Throttle on CPU
THROTTLE_RERANK_ON_CPU = True
RERANK_CANDIDATE_DOCS_CPU = 60
RERANKER_BATCH_SIZE_CPU   = 32

# Logging/Progress
LOG_LEVEL = "INFO"
SEARCH_LOG_EVERY = 16

# Output
SUBMIT_PATH = os.path.join(PROJECT_ROOT, "submit.csv")


# ---------------
# Logging setup
# ---------------
logger = logging.getLogger("run")
if not logger.handlers:
    hh = logging.StreamHandler(sys.stdout)
    hh.setFormatter(logging.Formatter("[%(levelname)s] %(message)s"))
    logger.addHandler(hh)
logger.setLevel(getattr(logging, LOG_LEVEL.upper(), logging.INFO))


# -------------------
# Robust CSV reading
# -------------------
def _bump_field_size_limit():
    max_int = sys.maxsize
    while True:
        try:
            csv.field_size_limit(max_int)
            break
        except OverflowError:
            max_int = int(max_int / 10)

def _read_csv_robust(path: str) -> pd.DataFrame:
    encs = ["utf-8", "utf-8-sig", "cp1251", "windows-1251", "latin1"]
    last = None
    for e in encs:
        try:
            return pd.read_csv(path, sep=",", engine="c", encoding=e, quoting=csv.QUOTE_MINIMAL)
        except Exception as ex:
            last = ex
    _bump_field_size_limit()
    for e in encs:
        try:
            return pd.read_csv(path, sep=None, engine="python", encoding=e, on_bad_lines="skip")
        except Exception as ex:
            last = ex
    try:
        with open(path, "r", encoding="utf-8", errors="ignore") as f:
            return pd.read_csv(StringIO(f.read()), sep=",", engine="c")
    except Exception:
        pass
    try:
        with open(path, "r", encoding="latin1", errors="ignore") as f:
            return pd.read_csv(StringIO(f.read()), sep=",", engine="c")
    except Exception:
        raise last if last else RuntimeError(f"Failed to read CSV: {path}")


# -----------------------
# Preprocessor & chunks
# -----------------------
@dataclass
class Chunk:
    chunk_id: str
    web_id: int
    document_id: int
    url: str
    title: str
    text: str
    chunk_idx: int
    is_title_chunk: bool

def _clean_text(t: str) -> str:
    if not isinstance(t, str): return ""
    return re.sub(r"\s+", " ", t).strip()

def _rename_like(df: pd.DataFrame) -> pd.DataFrame:
    low = {c: c.strip().lower() for c in df.columns}
    df = df.rename(columns=low)

    def pick(cols, candidates):
        for cand in candidates:
            if cand in cols:
                return cand
        return None

    cols = set(df.columns)
    # websites
    if "web_id" not in cols:
        alt = pick(cols, ["id", "page_id", "doc_id"])
        if alt: df = df.rename(columns={alt: "web_id"})
    if "url" not in cols:
        alt = pick(cols, ["link", "href"])
        if alt: df = df.rename(columns={alt: "url"})
    if "title" not in cols:
        alt = pick(cols, ["page_title", "name", "header"])
        if alt: df = df.rename(columns={alt: "title"})
    if "text" not in cols:
        alt = pick(cols, ["content", "body", "parsed_text", "html_text"])
        if alt: df = df.rename(columns={alt: "text"})
    # questions
    if "q_id" not in cols:
        alt = pick(cols, ["id", "question_id"])
        if alt: df = df.rename(columns={alt: "q_id"})
    if "query" not in cols:
        alt = pick(cols, ["question", "q_text", "text"])
        if alt: df = df.rename(columns={alt: "query"})
    return df

from transformers import AutoTokenizer
class TokenChunker:
    def __init__(self, model_name: str):
        self.tokenizer = AutoTokenizer.from_pretrained(
            model_name, use_fast=True, trust_remote_code=TRUST_REMOTE_CODE
        )
        self.chunk_tokens = CHUNK_TOKENS
        self.overlap_tokens = OVERLAP_TOKENS

    def chunk_text(self, text: str) -> List[str]:
        if not text: return []
        tokens = self.tokenizer.encode(text, add_special_tokens=False)
        chunks, step = [], (self.chunk_tokens - self.overlap_tokens)
        for start in range(0, len(tokens), step):
            piece = tokens[start:start+self.chunk_tokens]
            if not piece: break
            chunks.append(self.tokenizer.decode(piece, skip_special_tokens=True))
            if start + self.chunk_tokens >= len(tokens):
                break
        return chunks

def load_corpus() -> pd.DataFrame:
    df = _read_csv_robust(WEBSITES_CSV)
    df = _rename_like(df)
    need = ["web_id", "url", "title", "text"]
    for col in need:
        if col not in df.columns:
            if col in ("title", "text"):
                df[col] = ""
            else:
                raise ValueError(f"websites CSV missing column '{col}'")
        else:
            df[col] = df[col].fillna("")
    return df[need]

def load_queries() -> pd.DataFrame:
    dfq = _read_csv_robust(QUESTIONS_CSV)
    dfq = _rename_like(dfq)
    if "q_id" not in dfq.columns or "query" not in dfq.columns:
        raise ValueError("questions CSV must contain q_id and query")
    dfq["query"] = dfq["query"].fillna("")
    return dfq[["q_id", "query"]]

def create_chunks_from_websites(df_web: pd.DataFrame, model_name: str) -> List[Chunk]:
    tk = TokenChunker(model_name)
    chunks: List[Chunk] = []
    for row in df_web.itertuples(index=False):
        web_id = int(getattr(row, "web_id"))
        url   = _clean_text(getattr(row, "url", ""))
        title = _clean_text(getattr(row, "title", ""))
        body  = _clean_text(getattr(row, "text", ""))

        lead = body[:500] if body else ""
        title_payload = (title + " " + lead).strip() if (title or lead) else ""
        idx = 0
        if title_payload:
            chunks.append(Chunk(
                chunk_id=f"{web_id}::t0", web_id=web_id, document_id=web_id,
                url=url, title=title, text=title_payload, chunk_idx=idx, is_title_chunk=True
            ))
            idx += 1
        for piece in tk.chunk_text(body):
            chunks.append(Chunk(
                chunk_id=f"{web_id}::{idx}", web_id=web_id, document_id=web_id,
                url=url, title=title, text=piece, chunk_idx=idx, is_title_chunk=False
            ))
            idx += 1
    return chunks


# -----------------
# Embedder (GPU)
# -----------------
from sentence_transformers import SentenceTransformer

_E5_LIKE_PATTERNS = [r"\be5\b", r"\bgte\b", r"\bbge-m3\b", r"\b(text-)?embedding\b"]
def _is_e5_like(model_name: str) -> bool:
    name = model_name.lower()
    return any(re.search(p, name) for p in _E5_LIKE_PATTERNS)

class Embedder:
    def __init__(self, model_name: Optional[str] = None, normalize: Optional[bool] = None):
        self.model_name = model_name or EMBEDDING_MODEL
        self.normalize  = EMBEDDING_NORMALIZE if normalize is None else normalize
        self._use_e5_prefix = _is_e5_like(self.model_name)
        self.model = SentenceTransformer(self.model_name, trust_remote_code=TRUST_REMOTE_CODE, device=None)

    def _maybe_prefix_queries(self, queries: List[str]) -> List[str]:
        if not self._use_e5_prefix: return queries
        return [f"query: {q}" for q in queries]

    def _maybe_prefix_passages(self, passages: List[str]) -> List[str]:
        if not self._use_e5_prefix: return passages
        return [f"passage: {p}" for p in passages]

    def encode_queries(self, queries: List[str], batch_size: int = QUERY_EMB_BATCH_SIZE) -> np.ndarray:
        texts = self._maybe_prefix_queries(queries)
        emb = self.model.encode(texts, batch_size=batch_size, convert_to_numpy=True,
                                show_progress_bar=False, normalize_embeddings=self.normalize)
        return emb.astype(np.float32)

    def encode_passages(self, passages: List[str], batch_size: int = 256) -> np.ndarray:
        texts = self._maybe_prefix_passages(passages)
        emb = self.model.encode(texts, batch_size=batch_size, convert_to_numpy=True,
                                show_progress_bar=True, normalize_embeddings=self.normalize)
        return emb.astype(np.float32)


# ---------------
# BM25 over chunks
# ---------------
from rank_bm25 import BM25Okapi
try:
    import snowballstemmer  # type: ignore
    _STEMMER = snowballstemmer.stemmer("russian") if BM25_STEMMING else None
except Exception:
    _STEMMER = None

_WORD_RE = re.compile(r"[A-Za-zА-Яа-яЁё0-9]+")

def _normalize_token(t: str) -> str:
    t = t.lower()
    if BM25_NORMALIZE_E:
        t = t.replace("ё", "е")
    return t

def _stem(t: str) -> str:
    if not _STEMMER: return t
    return _STEMMER.stemWord(t)

def _tokenize_ru(text: str) -> List[str]:
    if not text: return []
    toks = _WORD_RE.findall(text)
    out = []
    for t in toks:
        tt = _normalize_token(t)
        if len(tt) < BM25_MIN_TOKEN_LEN: continue
        out.append(_stem(tt))
    return out

def _expand_query_tokens(tokens: List[str]) -> List[str]:
    if not BM25_SYNONYMS: return tokens
    bag = set(tokens)
    for canon, variants in BM25_SYNONYMS.items():
        all_forms = [canon] + variants
        norm_forms = [_stem(_normalize_token(x)) for x in all_forms]
        if any(f in bag for f in norm_forms):
            bag.update(norm_forms)
    return list(bag)

@dataclass
class _ChunkLite:
    chunk_id: str
    text: str

class BM25ChunkIndex:
    def __init__(self, tokenized_corpus: List[List[str]], chunks: List[_ChunkLite]):
        self._bm25 = BM25Okapi(tokenized_corpus)
        self._chunk_ids = [c.chunk_id for c in chunks]

    @classmethod
    def build_from_chunks(cls, chunks_full: Iterable[Chunk]) -> "BM25ChunkIndex":
        chunks = [_ChunkLite(c.chunk_id, c.text) for c in chunks_full]
        tokenized = [_tokenize_ru(c.text) for c in chunks]
        return cls(tokenized, chunks)

    def query(self, query: str, top_k: int = 100) -> List[Tuple[str, float]]:
        q_tokens = _expand_query_tokens(_tokenize_ru(query))
        scores = self._bm25.get_scores(q_tokens)
        order = np.argsort(scores)[::-1][:top_k]
        return [(self._chunk_ids[int(i)], float(scores[int(i)])) for i in order]

    def save(self, path: str) -> None:
        with open(path, "wb") as f:
            pickle.dump({"bm25": self._bm25, "chunk_ids": self._chunk_ids}, f)

    @classmethod
    def load(cls, path: str) -> "BM25ChunkIndex":
        with open(path, "rb") as f:
            obj = pickle.load(f)
        inst = object.__new__(cls)
        inst._bm25 = obj["bm25"]
        inst._chunk_ids = obj["chunk_ids"]
        return inst


# ------------------------
# FAISS dense index/search
# ------------------------
def _build_faiss(emb_dim: int):
    try:
        import faiss
    except Exception as e:
        logger.warning(f"FAISS not available, falling back to NumPy search ({e})")
        return None, None

    use_gpu = False
    if not FAISS_CPU:
        # Try GPU
        try:
            res = faiss.StandardGpuResources()
            use_gpu = True
        except Exception:
            use_gpu = False

    index = faiss.IndexFlatIP(emb_dim)
    if use_gpu:
        try:
            index = faiss.index_cpu_to_all_gpus(index)
            logger.info("FAISS: using GPU index")
        except Exception:
            logger.info("FAISS: GPU transfer failed, using CPU")
    else:
        logger.info("FAISS: using CPU index")

    return index, use_gpu


# --------------------
# Cross-Encoder CE
# --------------------
import torch
from sentence_transformers import CrossEncoder

from FlagEmbedding import FlagLLMReranker

class CrossEncoderReranker:
    def __init__(self, prefer: Optional[str] = None):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        # здесь prefer/RERANKER_MODEL можно игнорировать, жёстко берём gemma
        self.model = FlagLLMReranker(
            "BAAI/bge-reranker-v2-gemma",
            use_fp16=True  # на GPU, чтобы не умереть по времени/памяти
        )
        logger.info(f"[RERANKER] loaded: BAAI/bge-reranker-v2-gemma on {device}")

    def score(self, query: str, docs: List[str], batch_size: Optional[int] = None) -> List[float]:
        if not docs:
            return []
        pairs = [[query, d] for d in docs]
        # FlagLLMReranker сам батает пары внутри, batch_size можно игнорировать
        scores = self.model.compute_score(pairs)  # список float
        return [float(s) for s in scores]



# -------------------
# Retrieval pipeline
# -------------------
def re_split_simple(text: str) -> List[str]:
    return _WORD_RE.findall(text or "")

def _min_max_norm(scores: Dict[int, float]) -> Dict[int, float]:
    if not scores: return {}
    vals = list(scores.values())
    vmin, vmax = min(vals), max(vals)
    if math.isclose(vmin, vmax): return {k: 0.0 for k in scores}
    return {k: (v - vmin) / (vmax - vmin) for k, v in scores.items()}

_POS_PATTS = [re.compile(p, re.I) for p in URL_POSITIVE_PATTERNS]
_NEG_PATTS = [re.compile(p, re.I) for p in URL_NEGATIVE_PATTERNS]
def _pos_url_title_bonus(url: str, title: str) -> float:
    s = url + " " + (title or "")
    return URL_POSITIVE_BOOST if any(p.search(s) for p in _POS_PATTS) else 0.0
def _neg_url_penalty(url: str) -> float:
    return URL_NEGATIVE_PENALTY if any(p.search(url) for p in _NEG_PATTS) else 0.0

def _is_short_query(q: str) -> bool:
    toks = [t for t in re_split_simple(q) if len(t) > 2]
    return len(toks) <= SHORT_QUERY_TOKENS
def _has_digits(q: str) -> bool:
    return any(ch.isdigit() for ch in q)
def _has_keywords(q: str) -> bool:
    low = q.lower()
    return any(kw in low for kw in BM25_BONUS_KEYWORDS)

def _dynamic_weights(q: str) -> Tuple[float, float]:
    d, b = WEIGHT_DENSE, WEIGHT_BM25
    if not USE_DYNAMIC_WEIGHTS:
        return d, b
    bonus = 0.0
    if _is_short_query(q): bonus += BM25_BONUS_SHORT
    if _has_digits(q):     bonus += BM25_BONUS_HAS_DIGITS
    if _has_keywords(q):   bonus += 0.15
    b_new = min(0.9, max(0.1, b + bonus))
    d_new = 1.0 - b_new
    return d_new, b_new

def _sentence_split(text: str) -> List[str]:
    parts = re.split(r"(?<=[\.\!\?])\s+", text)
    return [p.strip() for p in parts if p and p.strip()]

def _sent_score(sent: str, q_tokens: set[str]) -> float:
    toks = {t.lower() for t in re_split_simple(sent) if len(t) > 2}
    return float(len(toks & q_tokens))

class OneFileRetrieval:
    def __init__(self):
        self.embedder = Embedder()
        self._bm25: Optional[BM25ChunkIndex] = None
        self._faiss_index = None
        self._faiss_use_gpu = None
        self._chunk_meta: Dict[str, Chunk] = {}
        self._doc_info: Dict[int, Tuple[str, str]] = {}
        self._chunk_ids: List[str] = []  # order matches FAISS ids
        self._reranker = CrossEncoderReranker() if USE_RERANKER else None

    def build_index(self) -> None:
        logger.info("Loading corpus...")
        df_web = load_corpus()
        logger.info(f"Corpus: {len(df_web)} pages")
        logger.info("Chunking pages (token-based)...")
        chunks = create_chunks_from_websites(df_web, model_name=EMBEDDING_MODEL)
        logger.info(f"Total chunks: {len(chunks)}")

        self._chunk_meta = {c.chunk_id: c for c in chunks}
        for c in chunks:
            if c.document_id not in self._doc_info:
                self._doc_info[c.document_id] = (c.url, c.title)

        # Dense embeddings for chunks (GPU)
        logger.info("Encoding chunk embeddings (dense)...")
        docs_all = [c.text for c in chunks]
        emb_all = []
        t0 = time.time()
        for start in range(0, len(docs_all), BATCH_ADD_SIZE):
            end = min(start + BATCH_ADD_SIZE, len(docs_all))
            emb = self.embedder.encode_passages(docs_all[start:end], batch_size=256)
            emb_all.append(emb)
            logger.info(f"  chunks {start}..{end-1} done (elapsed {time.time()-t0:.1f}s)")
        emb_all = np.vstack(emb_all).astype(np.float32)
        logger.info(f"Embeddings ready: {emb_all.shape} in {time.time()-t0:.1f}s")

        # FAISS index (cosine via IP on normalized)
        self._faiss_index, self._faiss_use_gpu = _build_faiss(emb_all.shape[1])
        if self._faiss_index is None:
            # fallback: store for NumPy search
            self._emb_matrix = emb_all
            logger.info("Using NumPy dense search fallback.")
        else:
            self._faiss_index.add(emb_all)
        self._chunk_ids = [c.chunk_id for c in chunks]
        del emb_all; gc.collect()

        # BM25 over chunks
        if USE_BM25:
            logger.info("Building BM25 index over chunks...")
            self._bm25 = BM25ChunkIndex.build_from_chunks(chunks)
            self._bm25.save(BM25_INDEX_PATH)
            logger.info(f"BM25 saved to {BM25_INDEX_PATH}")

        logger.info("Index build completed.")

    # Dense candidates for a batch of queries
    def _dense_candidates_batch(self, q_vecs: np.ndarray, n_results: int) -> List[List[Tuple[str, float]]]:
        if getattr(self, "_faiss_index", None) is None:
            # NumPy fallback
            sims = np.dot(q_vecs, self._emb_matrix.T)  # (B, N)
            idx = np.argsort(-sims, axis=1)[:, :n_results]
            out = []
            for i in range(q_vecs.shape[0]):
                ids = idx[i]
                pairs = [(self._chunk_ids[j], float(sims[i, j])) for j in ids]
                out.append(pairs)
            return out

        # FAISS path
        import faiss  # type: ignore
        D, I = self._faiss_index.search(q_vecs, n_results)  # sims & indices
        out = []
        for i in range(q_vecs.shape[0]):
            pairs = []
            for d, j in zip(D[i], I[i]):
                if j < 0: continue
                pairs.append((self._chunk_ids[int(j)], float(d)))
            out.append(pairs)
        return out

    def _topk_pool_per_doc(
        self, pairs: List[Tuple[str, float]], k: int
    ) -> Tuple[Dict[int, float], Dict[int, List[Tuple[str, float]]]]:
        per_doc: Dict[int, List[Tuple[str, float]]] = {}
        for chunk_id, score in pairs:
            ch = self._chunk_meta.get(chunk_id)
            if not ch: continue
            per_doc.setdefault(ch.document_id, []).append((chunk_id, float(score)))
        doc_scores: Dict[int, float] = {}
        doc_chunks: Dict[int, List[Tuple[str, float]]] = {}
        for doc_id, vals in per_doc.items():
            vals.sort(key=lambda x: x[1], reverse=True)
            pool = vals[:k]
            if pool:
                smax = pool[0][1]
                mean_k = float(np.mean([s for _, s in pool]))
                doc_scores[doc_id] = float(smax + 0.5 * mean_k)
                doc_chunks[doc_id] = pool
            else:
                doc_scores[doc_id] = 0.0
                doc_chunks[doc_id] = []
        return doc_scores, doc_chunks

    def _apply_domain_boosts(self, scores: Dict[int, float], q_text: str) -> Dict[int, float]:
        if not scores: return scores
        out = {}
        q_low = q_text.lower()
        is_premium = any(x in q_low for x in ["a-club", "аклуб", "премиум", "private", "wealth"])
        for doc_id, s in scores.items():
            url, title = self._doc_info.get(doc_id, ("", ""))
            bonus = _pos_url_title_bonus(url, title)
            malus = 0.0 if is_premium else _neg_url_penalty(url)
            out[doc_id] = s * (1.0 + bonus) * (1.0 - malus)
        return out

    def _build_doc_text_for_ce(
        self, q_text: str, doc_id: int, dense_chunks: Dict[int, List[Tuple[str, float]]],
        bm25_chunks: Dict[int, List[Tuple[str, float]]]
    ) -> str:
        title = self._doc_info.get(doc_id, ("", ""))[1] or ""
        merged: List[Tuple[str, float]] = []
        for dct in (dense_chunks, bm25_chunks):
            if doc_id in dct:
                merged.extend(dct[doc_id])
        seen = set()
        merged = sorted(merged, key=lambda x: x[1], reverse=True)
        uniq: List[Tuple[str, float]] = []
        for cid, s in merged:
            if cid not in seen:
                uniq.append((cid, s)); seen.add(cid)

        parts: List[str] = []
        q_tokens = {t.lower() for t in re_split_simple(q_text) if len(t) > 2}
        for cid, _ in uniq[:RERANKER_CHUNKS_PER_DOC]:
            ch = self._chunk_meta.get(cid)
            if not ch: continue
            sents = _sentence_split(ch.text)
            scored = sorted(sents, key=lambda st: _sent_score(st, q_tokens), reverse=True)
            take = max(1, min(3, len(scored)))
            parts.extend(scored[:take])

        text_blob = (title + "\n\n" + "\n".join(parts)).strip()
        if len(text_blob) > RERANKER_MAX_CHARS_PER_DOC:
            text_blob = text_blob[:RERANKER_MAX_CHARS_PER_DOC]
        return text_blob

    def search_batch(self, queries: List[str], top_k: int) -> List[List[int]]:
        if not queries: return []
        t0 = time.time()
        logger.info(f"[search_batch] start: {len(queries)} queries")

        # Encode queries
        qq = self.embedder.encode_queries(queries, batch_size=QUERY_EMB_BATCH_SIZE)
        logger.info(f"[search_batch] embedded {len(queries)} q in {time.time()-t0:.2f}s")

        # Dense candidates
        t1 = time.time()
        dense_pairs_list = self._dense_candidates_batch(qq, n_results=SEARCH_TOP_K_CHUNKS_DENSE)
        logger.info(f"[search_batch] dense candidates in {time.time()-t1:.2f}s")

        # Rerank throttling on CPU
        local_rerank_k = RERANK_CANDIDATE_DOCS
        local_bs = RERANKER_BATCH_SIZE
        if THROTTLE_RERANK_ON_CPU and not torch.cuda.is_available():
            local_rerank_k = min(local_rerank_k, RERANK_CANDIDATE_DOCS_CPU)
            local_bs = max(1, RERANKER_BATCH_SIZE_CPU)
            logger.info(f"[search_batch] CPU detected → cand={local_rerank_k}, bs={local_bs}")

        results = []
        processed = 0
        every = max(1, SEARCH_LOG_EVERY)

        for i, (q_text, pairs) in enumerate(zip(queries, dense_pairs_list), 1):
            t_q = time.time()
            w_dense, w_bm25 = _dynamic_weights(q_text)

            dense_doc, dense_chunks = self._topk_pool_per_doc(pairs, k=AGG_TOP_K_PER_DOC)

            bm25_doc: Dict[int, float] = {}
            bm25_chunks: Dict[int, List[Tuple[str, float]]] = {}
            if USE_BM25 and self._bm25 is not None:
                add = 150 if (_is_short_query(q_text) or _has_keywords(q_text)) else 0
                bm25_pairs = self._bm25.query(q_text, top_k=SEARCH_TOP_K_CHUNKS_BM25 + add)
                bm25_doc, bm25_chunks = self._topk_pool_per_doc(bm25_pairs, k=AGG_TOP_K_PER_DOC)

            dense_norm = _min_max_norm(dense_doc)
            bm25_norm  = _min_max_norm(bm25_doc) if USE_BM25 else {}
            all_docs   = set(dense_norm.keys()) | set(bm25_norm.keys())
            hybrid_scores = {
                d: (w_dense * dense_norm.get(d, 0.0) + w_bm25 * bm25_norm.get(d, 0.0))
                for d in all_docs
            }

            if TITLE_BOOST > 0:
                q_tokens = {t.lower() for t in re_split_simple(q_text) if len(t) > 2}
                for doc_id, (_, title) in self._doc_info.items():
                    title_tokens = {t.lower() for t in re_split_simple(title) if len(t) > 2}
                    if q_tokens & title_tokens and doc_id in hybrid_scores:
                        hybrid_scores[doc_id] *= (1.0 + TITLE_BOOST)

            hybrid_scores = self._apply_domain_boosts(hybrid_scores, q_text)
            prelim_ranked = sorted(hybrid_scores.items(), key=lambda x: x[1], reverse=True)
            prelim_docs = [doc for doc, _ in prelim_ranked]

            # Cross-Encoder rerank
            if USE_RERANKER and (self._reranker is not None):
                cand_docs = prelim_docs[:local_rerank_k]
                cand_texts = [self._build_doc_text_for_ce(q_text, d, dense_chunks, bm25_chunks) for d in cand_docs]
                ce_scores = self._reranker.score(q_text, cand_texts, batch_size=local_bs)
                if ce_scores:
                    ce_min, ce_max = float(min(ce_scores)), float(max(ce_scores))
                    ce_norm = [0.0 if math.isclose(ce_min, ce_max) else (s - ce_min) / (ce_max - ce_min) for s in ce_scores]
                    mix_scores: Dict[int, float] = {}
                    for d, base_s, ce_s in zip(cand_docs, [hybrid_scores.get(d, 0.0) for d in cand_docs], ce_norm):
                        mix_scores[d] = (1.0 - RERANKER_ALPHA) * base_s + RERANKER_ALPHA * ce_s
                    final_ranked = sorted(mix_scores.items(), key=lambda x: x[1], reverse=True)
                    top_docs = [doc for doc, _ in final_ranked[:top_k]]
                else:
                    top_docs = prelim_docs[:top_k]
            else:
                top_docs = prelim_docs[:top_k]

            # pad/unique to top_k
            if len(top_docs) < top_k:
                seen = set(top_docs)
                for d in prelim_docs:
                    if d not in seen:
                        top_docs.append(d); seen.add(d)
                    if len(top_docs) == top_k: break

            uniq_docs = []
            for d in top_docs:
                if d not in uniq_docs:
                    uniq_docs.append(d)
                if len(uniq_docs) == top_k: break

            results.append(uniq_docs)
            processed += 1

            if processed == 1 or processed % every == 0 or processed == len(queries):
                elapsed = time.time() - t0
                rate = processed / max(elapsed, 1e-6)
                eta  = (len(queries) - processed) / max(rate, 1e-6)
                logger.info(f"[search_batch] {processed}/{len(queries)} | last {time.time()-t_q:.2f}s | "
                            f"avg {elapsed/processed:.2f}s/q | ETA {eta/60:.1f} min")

        return results

    def batch_answer(self, dfq: pd.DataFrame) -> pd.DataFrame:
        n = len(dfq)
        if n == 0:
            return pd.DataFrame(columns=["q_id", "web_list"])
        logger.info(f"[batch_answer] Start answering {n} queries...")
        t0 = time.time()
        rows = []
        chunk = max(1, BATCH_ANSWER_CHUNK)
        for i_start in range(0, n, chunk):
            i_end = min(i_start + chunk, n)
            part = dfq.iloc[i_start:i_end]
            queries = part["query"].tolist()
            q_ids   = part["q_id"].astype(int).tolist()
            t_chunk = time.time()
            docs_lists = self.search_batch(queries, top_k=FINAL_TOP_K_DOCS)
            dur = time.time() - t_chunk
            for q_id, docs in zip(q_ids, docs_lists):
                while len(docs) < FINAL_TOP_K_DOCS:
                    docs.append(docs[-1] if docs else 1)
                rows.append({"q_id": q_id, "web_list": docs})
            done = i_end
            elapsed = time.time() - t0
            rate = done / max(elapsed, 1e-6)
            eta  = (n - done) / max(rate, 1e-6)
            logger.info(f"[batch_answer] {done}/{n} | chunk {i_end-i_start} in {dur:.2f}s | "
                        f"avg {elapsed/done:.2f}s/q | {rate:.2f} q/s | ETA {eta/60:.1f} min")
        logger.info(f"[batch_answer] Finished {n} queries in {time.time()-t0:.1f}s")
        return pd.DataFrame(rows)


# -----------
# Main
# -----------
def main():
    print("PROJECT_ROOT =", PROJECT_ROOT, flush=True)
    print("WEBSITES_CSV =", WEBSITES_CSV, flush=True)
    print("QUESTIONS_CSV =", QUESTIONS_CSV, flush=True)

    retr = OneFileRetrieval()
    t0 = time.time()
    retr.build_index()
    print(f"[INFO] Index ready in {time.time()-t0:.1f}s. Loading queries...", flush=True)

    dfq = load_queries()
    print(f"[INFO] Queries loaded: {len(dfq)}. Starting answering...", flush=True)

    t1 = time.time()
    out = retr.batch_answer(dfq)
    print(f"[INFO] Answering done in {time.time()-t1:.1f}s. Saving submit.csv ...", flush=True)

    out["web_list"] = out["web_list"].apply(lambda xs: json.dumps(xs, ensure_ascii=False))
    out.to_csv(SUBMIT_PATH, index=False)
    print(f"submit.csv saved → {SUBMIT_PATH}", flush=True)

if __name__ == "__main__":
    main()

2025-11-14 14:08:28.465318: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763129308.658507      48 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763129308.709071      48 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...

[INFO] [search_batch] embedded 128 q in 0.61s


INFO:run:[search_batch] embedded 128 q in 0.61s


[INFO] [search_batch] dense candidates in 0.03s


INFO:run:[search_batch] dense candidates in 0.03s
pre tokenize: 100%|██████████| 2/2 [00:00<00:00, 188.06it/s]
You're using a GemmaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
 33%|███▎      | 1/3 [00:12<00:24, 12.06s/it]


OutOfMemoryError: CUDA out of memory. Tried to allocate 54.00 MiB. GPU 0 has a total capacity of 15.89 GiB of which 53.12 MiB is free. Process 4014 has 15.83 GiB memory in use. Of the allocated memory 15.47 GiB is allocated by PyTorch, and 83.65 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)